## import libraries

In [12]:
import os, glob
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras

## image resize

In [10]:
def image_resize():
    img_path = os.getenv("HOME") + "/aiffel_practice/rock_scissor_paper/"
    img_lst = ["scissor", "rock", "paper"]
    target_size = (28, 28)

    for i, val in enumerate(img_lst):
        imgs = glob.glob(img_path+ val +"/*.jpg")
        for img in imgs:
            old_img = Image.open(img)
            new_img = old_img.resize(target_size, Image.ANTIALIAS)
            new_img.save(img, "JPEG")

image_resize()

## load resized images

In [24]:
def load_data(img_path):
    num_data = 300
    img_size = 28
    channel = 3
    imgs = np.zeros(num_data * img_size * img_size * channel, dtype=np.int32)
    imgs = imgs.reshape(num_data, img_size, img_size, channel)
    labels = np.zeros(num_data, dtype=np.int32)
    
    img_lst = ["scissor", "rock", "paper"]
    target_size = (28, 28)
    label_idx = 0
    
    for i, val in enumerate(img_lst):
        imgs_path = glob.glob(img_path+ val +"/*.jpg")
        for img in imgs_path:
            img = np.array(Image.open(img), dtype=np.int32)
            imgs[label_idx, :, :, :] = img
            labels[label_idx] = i
            label_idx = label_idx + 1
    return imgs, labels

def img_normalize(imgs):
    return imgs/255.0

img_path = os.getenv("HOME") + "/aiffel_practice/rock_scissor_paper/"
(x_train, y_train) = load_data(img_path)
x_train_norm = img_normalize(x_train)

## design deep learning model

In [17]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3, 3), activation="relu", input_shape=(28, 28, 3)))
model.add(keras.layers.MaxPool2D(2, 2))
model.add(keras.layers.Conv2D(32, (3, 3), activation="relu"))
model.add(keras.layers.MaxPool2D(2, 2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation="relu"))
model.add(keras.layers.Dense(3, activation="softmax"))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 16)        448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                25632     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                

### learning model

In [26]:
model.compile(optimizer="adam",
               loss="sparse_categorical_crossentropy",
                metrics=["accuracy"])
model.fit(x_train_norm, y_train, epochs = 20)

Epoch 1/20
10/10 [==============================] - 3s 346ms/step - loss: 1.0837 - accuracy: 0.3667
Epoch 2/20
10/10 [==============================] - 0s 2ms/step - loss: 1.0087 - accuracy: 0.5200
Epoch 3/20
10/10 [==============================] - 0s 2ms/step - loss: 0.8926 - accuracy: 0.7300
Epoch 4/20
10/10 [==============================] - 0s 2ms/step - loss: 0.7765 - accuracy: 0.7100
Epoch 5/20
10/10 [==============================] - 0s 2ms/step - loss: 0.5808 - accuracy: 0.8133
Epoch 6/20
10/10 [==============================] - 0s 2ms/step - loss: 0.4361 - accuracy: 0.8600
Epoch 7/20
10/10 [==============================] - 0s 2ms/step - loss: 0.3733 - accuracy: 0.8733
Epoch 8/20
10/10 [==============================] - 0s 2ms/step - loss: 0.2769 - accuracy: 0.9267
Epoch 9/20
10/10 [==============================] - 0s 2ms/step - loss: 0.2410 - accuracy: 0.9400
Epoch 10/20
10/10 [==============================] - 0s 2ms/step - loss: 0.2029 - accuracy: 0.9400
Epoch 11/20
10/10